In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2

In [2]:
train_dir='../input/emergency-vehicles-identification/Emergency_Vehicles/train'
df_train=pd.read_csv('../input/emergency-vehicles-identification/Emergency_Vehicles/train.csv')
df_test=pd.read_csv('../input/emergency-vehicles-identification/Emergency_Vehicles/test.csv')
df_train.head()

In [3]:
im=cv2.imread('../input/emergency-vehicles-identification/Emergency_Vehicles/train/1.jpg')
plt.imshow(im)

In [4]:
df_train['emergency_or_not']=df_train['emergency_or_not'].astype(str)

In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255)

In [6]:
train_generator=datagen.flow_from_dataframe(dataframe=df_train[:1150], directory=train_dir,x_col='image_names', y_col='emergency_or_not', class_mode='binary',target_size=(224,224),
                                            batch_size=150)
validation_generator=datagen.flow_from_dataframe(dataframe=df_train[1150:], directory=train_dir, x_col='image_names', y_col='emergency_or_not', class_mode='binary', target_size=(224,224), batch_size=150)

# MODEL BUILDING

In [7]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [8]:
mobile_net=MobileNetV2(weights='imagenet', input_shape=(224,224,3), include_top=False)

In [9]:
model=Sequential()
model.add(mobile_net)
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [10]:
#compiling the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit_generator(train_generator,epochs = 10,validation_data = validation_generator)

In [18]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']


plt.plot(acc, label=' Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

In [19]:
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [25]:
test_dir='../input/emergency-vehicles-identification/Emergency_Vehicles/test'

test_gen=ImageDataGenerator(rescale=1/255)

test_generator=test_gen.flow_from_dataframe(dataframe=df_test, directory=test_dir, x_col='image_names', class_mode=None, target_size=(224,224), batch_size=1, shuffle=False)

In [26]:
preds=model.predict_generator(test_generator, steps=len(test_generator.filenames))

In [27]:
preds

In [29]:
image_ids=[name.split('/')[-1] for name in test_generator.filenames]
predictions=preds.flatten()
data={'image_names':image_ids, 'emergency_or_not':predictions}
submission=pd.DataFrame(data)
submission.head()

In [30]:
submission['emergency_or_not']=submission['emergency_or_not'].apply(lambda x: 1 if x>0.75 else 0)

In [31]:
submission.to_csv('submission_mobilenet.csv', index=False)